# Apache Iceberg デモンストレーション

このノートブックでは、Apache Icebergの主要な機能を実際のコードを通じて学習します。

## 学習内容
1. **Time Travel**: スナップショット機能による履歴管理と過去データの参照
2. **メタデータ管理**: テーブル履歴とスナップショットの管理
3. **スキーマエボリューション**: カラムの追加・変更・削除の安全な実行

## 前提条件
- Unity Catalogが有効化されている環境
- パーソナル領域（users.{username}）へのアクセス権限
- Iceberg形式のテーブル作成権限


## 1. Time Travel（タイムトラベル）

Apache Icebergの**Time Travel**機能は、テーブルのスナップショット履歴を管理し、過去の状態を参照・復旧できる機能です。

### 主な用途
- **データの監査**: 特定時点でのデータ状態の確認
- **誤操作からの復旧**: 間違ったデータ変更からの復元
- **A/Bテスト**: 異なる時点のデータでの比較分析
- **レポート再現**: 過去のレポート結果の再現

### Icebergのスナップショット機能
- **スナップショット**: 各操作でテーブルの状態を記録
- **メタデータ管理**: 効率的なメタデータ構造で高速アクセス
- **タイムスタンプ/スナップショットID**: 複数の方法で過去データにアクセス可能


In [0]:
-- Unity Catalogのパーソナル領域を使用
-- 各ユーザーは自分のスキーマ名に変更してください
USE CATALOG users;
USE SCHEMA yukiteru_koide;  -- ← ここを自分のユーザー名に変更


### ステップ1: デモ用Icebergテーブルの作成と初期データ投入

Apache Icebergテーブルを作成し、初期データを投入します。この操作により最初のスナップショットが作成されます。


In [0]:
-- Apache Icebergテーブルを作成（USING ICEBERGを指定）
CREATE OR REPLACE TABLE iceberg_demo (
  id       INT,           -- 主キー
  amount   DECIMAL(10,2), -- 金額
  country  STRING,        -- 国コード
  category STRING,        -- 商品カテゴリ
  ts       TIMESTAMP      -- タイムスタンプ
) USING ICEBERG;

-- 初期データを投入（最初のスナップショットが作成される）
INSERT INTO iceberg_demo VALUES
  (1, 100.00, 'JP', 'Electronics', current_timestamp()),
  (2, 200.00, 'US', 'Books', current_timestamp()),
  (3, 150.00, 'JP', 'Clothing', current_timestamp()),
  (4, 300.00, 'UK', 'Electronics', current_timestamp());

-- 投入されたデータを確認
SELECT * FROM iceberg_demo ORDER BY id;


### ステップ2: 追加データの投入（新しいスナップショット作成）

新しいデータを追加してスナップショットを増やします。Icebergでは各操作が新しいスナップショットとして記録されます。


In [0]:
-- 新しいデータを追加してスナップショットを増やす
INSERT INTO iceberg_demo VALUES
  (5, 250.00, 'DE', 'Sports', current_timestamp()),
  (6, 180.00, 'FR', 'Books', current_timestamp());

-- 追加後のデータを確認
SELECT * FROM iceberg_demo ORDER BY id;


### ステップ3: データ更新操作（さらなるスナップショット作成）

既存データを更新して履歴を作成します。Icebergでは更新操作も新しいスナップショットとして記録されます。


### ステップ1: 履歴情報の確認

まず、テーブルの操作履歴とスナップショット情報を確認します。Icebergでは`DESCRIBE HISTORY`コマンドで詳細な履歴を参照できます。


In [ ]:
-- テーブルの履歴を確認
DESCRIBE HISTORY iceberg_demo;

-- 【履歴情報の項目】
-- ・made_current_at: スナップショットが作成された時刻
-- ・snapshot_id: 一意のスナップショット識別子
-- ・parent_id: 親スナップショットのID
-- ・is_current_ancestor: 現在のスナップショットの祖先かどうか


In [0]:
-- 既存データを更新して履歴を作成
UPDATE iceberg_demo 
SET amount = amount * 1.1 
WHERE country = 'JP';

-- 更新後のデータを確認（日本のデータが10%増加）
SELECT * FROM iceberg_demo WHERE country = 'JP' ORDER BY id;


### ステップ4: 誤操作のシミュレーション

**⚠️ 注意**: これは意図的な誤操作です！

Electronics カテゴリのデータを間違って削除してしまう状況を再現します。実際の運用では避けるべき操作ですが、復旧デモのために実行します。


In [0]:
-- Electronics カテゴリのデータを誤って削除
DELETE FROM iceberg_demo WHERE category = 'Electronics';

-- 削除後の状態を確認（Electronicsカテゴリが消えていることを確認）
SELECT * FROM iceberg_demo ORDER BY id;


## 2. メタデータ管理とスナップショット操作

Apache Icebergの**メタデータ管理**は、テーブルの履歴とスナップショットを効率的に管理する仕組みです。

### Icebergのメタデータ構造
- **スナップショット**: 各操作時点でのテーブル状態
- **メタデータファイル**: テーブルスキーマ、パーティション情報、統計情報
- **マニフェストファイル**: データファイルの場所と統計情報

### スナップショット管理の特徴
- **軽量な履歴管理**: メタデータのみで履歴を追跡
- **高速な時系列アクセス**: インデックス化された履歴情報
- **柔軟な保持ポリシー**: 必要に応じてスナップショットを削除可能

### Time Travelクエリの方法
- **タイムスタンプ指定**: `FOR SYSTEM_TIME AS OF`
- **スナップショットID指定**: `FOR SYSTEM_VERSION AS OF`


In [0]:
-- テーブル履歴を確認
DESCRIBE HISTORY iceberg_demo;


### ステップ2: タイムスタンプによるTime Travel

特定の時刻のデータを参照します（削除前の状態）。

**補足**: 現在時刻から相対的に指定することも可能
```sql
SELECT * FROM iceberg_demo FOR SYSTEM_TIME AS OF (current_timestamp() - INTERVAL 10 MINUTES);
```

※ 実際の時刻は上記の`DESCRIBE HISTORY`の結果を参考に調整してください


In [0]:
-- 特定の時刻のデータを参照
SELECT * FROM iceberg_demo 
FOR SYSTEM_TIME AS OF '2025-10-02T05:47:20.529+00:00'
ORDER BY id;


In [0]:
-- ### ステップ3: スナップショットIDによるTime Travel

-- 特定のスナップショットIDのデータを参照
SELECT * FROM iceberg_demo 
FOR SYSTEM_TIME AS OF '2025-10-02T05:47:20.529+00:00'
ORDER BY id;

-- 【スナップショットIDの利点】
-- ・正確な状態の指定が可能
-- ・タイムゾーンの影響を受けない
-- ・プログラムからの参照に適している


In [0]:
-- ### ステップ4: データの復旧（INSERT INTO ... SELECT）

-- Time Travelを使用して削除されたデータを復旧
-- 削除前のスナップショットから削除されたレコードを特定して再挿入
INSERT INTO iceberg_demo
SELECT * FROM iceberg_demo 
FOR SYSTEM_TIME AS OF '2025-10-02T05:47:20.529+00:00'
WHERE category = 'Electronics';

-- 復旧後のデータを確認
SELECT * FROM iceberg_demo ORDER BY id;

-- 【復旧方法の特徴】
-- ・部分的な復旧が可能
-- ・条件を指定して特定データのみ復元
-- ・複数のスナップショットからのデータ統合も可能


## 3. ストレージ最適化とパフォーマンス向上

Apache Icebergでは、Delta LakeのVACUUMやLiquid Clusteringに相当する機能を使用してストレージとクエリパフォーマンスを最適化できます。

### 主な最適化機能
- **スナップショット管理**: 古いスナップショットの削除と孤児ファイルのクリーンアップ
- **ファイル最適化**: 小さなファイルの統合とデータの再編成
- **書き込み順序**: WRITE ORDERによるデータの並び替えとスキャン効率の向上


### ステップ1: スナップショット履歴の確認（VACUUM前の状態）

まず、現在のスナップショット状況を確認します。これがDelta LakeのVACUUM前の状態確認に相当します。


In [ ]:
-- 現在のスナップショット履歴を確認
DESCRIBE HISTORY iceberg_demo;

-- テーブルの詳細情報を確認
DESCRIBE DETAIL iceberg_demo;

-- 【確認ポイント】
-- ・スナップショット数の増加
-- ・各操作による履歴の蓄積
-- ・ファイル数とサイズの状況


### ステップ2: スナップショット保持ポリシーの設定

Icebergでは、テーブルプロパティでスナップショットの保持期間を設定できます。これがDelta LakeのVACUUM設定に相当します。


In [ ]:
-- スナップショット保持期間を設定（デモ用に短く設定）
ALTER TABLE iceberg_demo SET TBLPROPERTIES (
  'history.expire.max-snapshot-age-ms' = '3600000'  -- 1時間（3600秒 * 1000ms）
);

-- 孤児ファイルの保持期間も設定
ALTER TABLE iceberg_demo SET TBLPROPERTIES (
  'gc.enabled' = 'true',
  'commit.retry.num-retries' = '3'
);

-- 設定を確認
SHOW TBLPROPERTIES iceberg_demo;

-- 【保持ポリシーの説明】
-- ・history.expire.max-snapshot-age-ms: スナップショット保持期間
-- ・gc.enabled: ガベージコレクション有効化
-- ・本番環境では通常7日間（604800000ms）以上を推奨


### ステップ3: ファイル最適化（OPTIMIZE相当）

Icebergでは、小さなファイルを統合してクエリパフォーマンスを向上させることができます。これがDelta LakeのOPTIMIZEに相当します。


In [ ]:
-- 最適化前のファイル状況を確認
SELECT 
  file_path,
  file_format,
  record_count,
  file_size_in_bytes
FROM iceberg_demo.files
LIMIT 10;

-- ファイル統合による最適化を実行
-- 注意: CALL文はDatabricksの環境によって利用可能性が異なります
-- CALL system.rewrite_data_files('iceberg_demo');

-- 代替方法: 全データを再書き込みして最適化
CREATE OR REPLACE TABLE iceberg_demo_optimized
USING ICEBERG
AS SELECT * FROM iceberg_demo ORDER BY category, id;

-- 最適化後の状況を確認
DESCRIBE DETAIL iceberg_demo_optimized;

-- 【最適化の効果】
-- ・小さなファイルの統合
-- ・読み取りパフォーマンスの向上
-- ・ストレージ効率の改善


### ステップ4: 書き込み順序による最適化（Liquid Clustering相当）

IcebergではWRITE ORDERを使用してデータの並び順を制御し、スキャン効率を向上させることができます。これがDelta LakeのLiquid Clusteringに近い効果を提供します。


In [ ]:
-- WRITE ORDERを設定してテーブルを再作成
CREATE OR REPLACE TABLE iceberg_demo_clustered (
  id INT,
  name STRING,
  category STRING,
  price DOUBLE
)
USING ICEBERG
TBLPROPERTIES (
  'write.distribution-mode' = 'hash',
  'write.fanout-enabled' = 'true'
);

-- カテゴリ別に並び替えてデータを挿入（クラスタリング効果）
INSERT INTO iceberg_demo_clustered
SELECT id, name, category, price 
FROM iceberg_demo 
ORDER BY category, id;

-- さらに多くのデータを追加してクラスタリング効果を確認
INSERT INTO iceberg_demo_clustered VALUES
  (101, 'Gaming Mouse', 'Electronics', 79.99),
  (102, 'Mechanical Keyboard', 'Electronics', 149.99),
  (103, 'Fiction Novel', 'Books', 12.99),
  (104, 'Science Textbook', 'Books', 89.99),
  (105, 'Running Shoes', 'Sports', 119.99),
  (106, 'Tennis Racket', 'Sports', 199.99);

-- クラスタリング効果を確認するクエリ
SELECT category, COUNT(*) as count, AVG(price) as avg_price
FROM iceberg_demo_clustered 
GROUP BY category
ORDER BY category;

-- 【Liquid Clustering相当の効果】
-- ・カテゴリ別のデータ局所性向上
-- ・範囲スキャンの効率化
-- ・ファイル内でのデータ並び順最適化


### ステップ5: 最適化効果の確認

最適化前後でのクエリパフォーマンスを比較して、効果を確認します。


In [ ]:
-- 最適化前のテーブルでのクエリ
SELECT 'Original Table' as table_type, category, COUNT(*) as count
FROM iceberg_demo 
WHERE category = 'Electronics'
GROUP BY category;

-- 最適化後のテーブルでのクエリ
SELECT 'Clustered Table' as table_type, category, COUNT(*) as count
FROM iceberg_demo_clustered 
WHERE category = 'Electronics'
GROUP BY category;

-- ファイル数とサイズの比較
SELECT 
  'Original' as table_type,
  COUNT(*) as file_count,
  SUM(file_size_in_bytes) as total_size
FROM iceberg_demo.files
UNION ALL
SELECT 
  'Clustered' as table_type,
  COUNT(*) as file_count,
  SUM(file_size_in_bytes) as total_size
FROM iceberg_demo_clustered.files;

-- 【パフォーマンス向上のポイント】
-- ・スキャンするファイル数の削減
-- ・データ局所性の向上
-- ・I/O効率の改善
-- ・クエリ実行時間の短縮


## まとめ

このノートブックでは、Apache Icebergの3つの主要機能を実際のコードで体験しました。

### 学習した内容
1. **Time Travel**: スナップショット機能による履歴管理
   - タイムスタンプとスナップショットIDでの過去データ参照
   - 柔軟な復旧方法（部分復旧、条件指定復旧）
   - 軽量なメタデータ管理による高速アクセス

2. **メタデータ管理**: 効率的な履歴とスナップショット管理
   - スナップショット、メタデータファイル、マニフェストファイルの構造
   - 詳細な操作履歴の追跡
   - 高速な時系列データアクセス

3. **ストレージ最適化**: Delta Lake相当の最適化機能
   - スナップショット保持ポリシー（VACUUM相当）
   - ファイル最適化（OPTIMIZE相当）
   - 書き込み順序による効率化（Liquid Clustering相当）

### Icebergの特徴的な利点
- **高いパフォーマンス**: 効率的なメタデータ管理による高速クエリ
- **強力な後方互換性**: スキーマ変更後も古いクエリが動作
- **柔軟な履歴管理**: タイムスタンプとスナップショットIDの両方をサポート
- **包括的な最適化**: ストレージとクエリの両面での最適化機能

### Delta Lakeとの比較ポイント
- **メタデータ構造**: Icebergはより詳細で構造化されたメタデータ
- **Time Travel**: 両者ともに強力だが、アクセス方法が異なる
- **最適化機能**: 類似の機能を異なるアプローチで実現
- **パフォーマンス**: 大規模データでの効率性

### 本番環境での注意点
- **スナップショット保持**: 適切な保持ポリシーの設定
- **パフォーマンス**: 大量のスナップショットによる影響の監視
- **メタデータ管理**: 定期的なメンテナンスの実施
- **最適化頻度**: データ更新パターンに応じた最適化スケジュール

### 次のステップ
- パーティショニング戦略の学習
- Icebergの高度な機能（Hidden Partitioning等）の探索
- 大規模データでのパフォーマンス最適化
- 本番環境でのモニタリングとメンテナンス戦略
